This is the main Sensor Kennel application that will 
identify, read and plot any sensor that's plugged in.

On the lower side:  Pin1 (left) Gnd, Pin3 3v3, Pin-3 (from right) SDA, Pin-5 SCL.

In [219]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [220]:
%sendtofile --quiet --source device_detection.py


Sent 35 lines (1068 bytes) to device_detection.py.


In [211]:
%sendtofile --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source SDOF_funcs.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --binary --source OLED_driver.py



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fa74762cb00, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [259]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext

import time
time.sleep(0.5)
from device_detection import IdentifyI2CDevice

devices = IdentifyI2CDevice(i2c)
devname = "nothing"
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    fbuff.fill(0)
    fbuff.text(devname,0,0,1)
    fbuff.text(devdesc,0,12,1)
    doublepixels()
    oledshow()
    time.sleep(2)
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        oledshow()

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist: %.1f"%d, 8, 20, 1)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    setupTSL561(i2c)
    setregtimings(True, 1)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()

if devname == "TMP102":
    while True:
        r = i2c.readfrom_mem(0x48, 0x00, 2)
        t = r[0] + r[1]/256
        fbuff.fill(0)
        fbuff.text("TMP102", 0, 0, 1)
        fbuff.text("temp:", 0, 23, 1)
        fatntext("%.03f"%t, 30, 40)
        oledshow()

if devname == "SDOF":
    from SDOF_funcs import SetupAccGyrMag, readvectorsensor
    SetupAccGyrMag(i2c)
    while True:
        fbuff.fill(0)
        x, y, z = readvectorsensor("a")
        fbuff.text("Acc:", 0, 0, 1)
        fbuff.text("%d"%x, 8, 8, 1)
        fbuff.text("%d"%y, 8, 16, 1)
        fbuff.text("%d"%z, 8, 24, 1)
        x, y, z = readvectorsensor("g")
        fbuff.text("gyr:", 64, 0, 1)
        fbuff.text("%d"%x, 72, 8, 1)
        fbuff.text("%d"%y, 72, 16, 1)
        fbuff.text("%d"%z, 72, 24, 1)
        x, y, z = readvectorsensor("c")
        fbuff.text("mag:", 40, 32, 1)
        fbuff.text("%d"%x, 48, 40, 1)
        fbuff.text("%d"%y, 48, 48, 1)
        fbuff.text("%d"%z, 48, 56, 1)
        time.sleep(0.1)
        oledshow()


Sent 86 lines (2480 bytes) to main.py.


Sent 42 lines (1800 bytes) to SDOF_funcs.py.


In [256]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.
